In [ ]:
import cv2
import numpy as np

Now we create our yolo model by help of cfg and final weight trained by darkn

In [ ]:
net = cv2.dnn.readNet('C:/Users/ANKIT/Documents/covid19 mask/yolov3_training_last.weights',
                      'C:/Users/ANKIT/Documents/covid19 mask/yolov3_testing.cfg')

In [ ]:
classes = []
with open("C:/Users/ANKIT/Documents/covid19 mask/classes.txt","r") as f:
    classes = f.read().splitlines()

In [ ]:
font = cv2.FONT_HERSHEY_SIMPLEX #it represent the front type and  It will show you how to draw different 
#style text on images using OpenCV library

In [ ]:
label2category = {'Wearing Mask':0,'without_mask':1,'mask_weared_incorrect':2}
category2color = {0:(0,255,0),1:(0,0,255),2:(0,0,255)}

In [ ]:
cap = cv2.VideoCapture(0)
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    height,width,_ = frame.shape
    
    
# getLayerNames(): Get the name of all layers of the network.

# getUnconnectedOutLayers(): Get the index of the output layers.

    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    

# construct a blob from the input image and then perform a forward
# pass of the YOLO object detector, giving us our bounding boxes and
# associated probabilities

# we need to normalize, scale and reshape this image to be suitable as an input to the neural network: we do this 
# by cv2.dnn.blobFromImage
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),swapRB=True, crop=False)
    net.setInput(blob)
    
# Now let's feed this image into the neural network to get the output predictions:
# by net.forward
    layerOutputs = net.forward(ln)
    boxes = []
    confidences = []
    class_ids = []
    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence>0.1:
                center_x = int(detection[0]*width)
                center_y = int(detection[1]*height)
                
                w = int(detection[2]*width)
                h = int(detection[3]*height)
                
                x = int(center_x-w/2)
                y = int(center_x-h/2)
                boxes.append([x, y, w, h])
                confidences.append((float(confidence)))
                class_ids.append(classId)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.1, 0.4) #this function help to filter out most relevent boxes from image
    # predicted by yolov3 architecture. cv2.dnn.MNSBoxes(boxes,confidences,confidence,threshold)
    print(indexes)
    if (len(indexes)>0):
        for i in indexes.flatten():
            print(i)
            x , y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            print(label)
            label1 = label2category[label]
            confidence = str(round(confidences[i],2))
            cv2.rectangle(frame, (x,y), (x+w, y+h), category2color[label1], 2)
            cv2.putText(frame, label + " " + confidence, (x, y+10), font, 1,category2color[label1], 2)# help to write text on video

    # Display the resulting frame
    cv2.imshow('Image', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()